In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/ifeatMethods/data/featvec/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<40:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=40
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

8

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(10)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 6.61 ms, sys: 4.21 ms, total: 10.8 ms
Wall time: 5.05 s


In [12]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [13]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [14]:
get_model_stats('Geary')

{'train_min': 0.8488372093023255,
 'train_mean': 0.9093023255813956,
 'train_max': 0.9418604651162791,
 'train_std': 0.025896578431767545,
 'test_min': 0.6551724137931034,
 'test_mean': 0.7448275862068965,
 'test_max': 0.8275862068965517,
 'test_std': 0.04415947749953688}

In [15]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [16]:
results_df = pd.DataFrame(results_dict).T

In [17]:
results_df.to_csv('../data/SimResults/iFeatResults.csv',index=True)

In [18]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
GAAC,0.651163,0.702326,0.755814,0.029508,0.448276,0.558621,0.689655,0.070331
CKSAAP,0.837209,0.888372,0.941860,0.030411,0.655172,0.727586,0.793103,0.044828
NMBroto,0.837209,0.895349,0.930233,0.027517,0.586207,0.706897,0.827586,0.067660
CTDD,0.872093,0.896512,0.918605,0.014194,0.620690,0.655172,0.689655,0.030842
TPC,0.813953,0.897674,0.930233,0.034020,0.655172,0.724138,0.793103,0.046263
SOCNumber,0.883721,0.908140,0.930233,0.014194,0.448276,0.672414,0.758621,0.084816
Geary,0.848837,0.909302,0.941860,0.025897,0.655172,0.744828,0.827586,0.044159
Moran,0.860465,0.913953,0.941860,0.023372,0.655172,0.731034,0.793103,0.045747
PAAC,0.906977,0.918605,0.941860,0.012738,0.517241,0.700000,0.827586,0.077183
DPC,0.860465,0.918605,0.953488,0.024939,0.620690,0.727586,0.827586,0.064419
